In [1]:
import pandas as pd
import numpy as np
import re
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from itertools import product
from scipy import stats

%matplotlib inline
plt.style.use('ggplot')
#plt.style.use('seaborn-pastel')
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

sys.path.append("..")
from utils.util_functions import *

In [2]:
event_df = make_df("20180101", "20181231")

In [3]:
# エントロピーの計算
# 観測データの１次元配列がインプット
def entropy(data):
    #epsilon = 1e-9
    #all_case_list = ["".join(map(str, case)) for case in product([0, 1, 2, 3], [0, 1], [0, 1], [0, 1])]
    count = pd.Series([tuple(elem) for elem in data]).value_counts()
    prob = count / sum(count)
    return - sum(prob * np.log(prob))

In [4]:
def cond_entropy(target_data, given_data):
    assert len(given_data.shape) == 2
    combined_data = [(target_elem, tuple(given_elem)) for target_elem, given_elem in zip(target_data, given_data)]
    combined_entropy = entropy(combined_data)
    single_entropy = entropy(given_data)
    return combined_entropy - single_entropy

In [5]:
#entropy(pd.DataFrame(lst)[2].values), cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)

In [ ]:
#entropy(pd.DataFrame(lst)[[2, 1]].values)

In [ ]:
#cond_entropy(pd.DataFrame(lst)[1].values, pd.DataFrame(lst)[[2]].values) #+ entropy(pd.DataFrame(lst)[2].values)

In [ ]:
#lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))

In [ ]:
# H(St+1 | St, St-1)
#cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values)

In [ ]:
# H(St+1 | St)
#cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)

In [ ]:
for team in pacific:
    lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))
    # H(St+1 | St, St-1)
    second_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values)
    # H(St+1 | St)
    first_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)
    print(team, second_cond_entropy, first_cond_entropy, second_cond_entropy - first_cond_entropy)

In [ ]:
for team in central:
    lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))
    # H(St+1 | St, St-1)
    second_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values)
    # H(St+1 | St)
    first_cond_entropy = cond_entropy(pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values)
    print(team, second_cond_entropy, first_cond_entropy, second_cond_entropy - first_cond_entropy)

In [ ]:
#event_df.head()

# work

In [ ]:
#target_data, given_data = pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[0, 1]].values
target_data, given_data = pd.DataFrame(lst)[2].values, pd.DataFrame(lst)[[1]].values
combined_data = [(target_elem, tuple(given_elem)) for target_elem, given_elem in zip(target_data, given_data)]

In [ ]:
#pd.Series(combined_data).value_counts()

In [ ]:
#combined_data

In [ ]:
pd.Series([(1, (1)), (1, (1))]).value_counts()

In [ ]:
for team in pacific:
    lst = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))
    print(team, entropy(lst))

In [ ]:
lst = make_flattened_list(make_inning_list(event_df[event_df.攻撃チーム==team]))

In [ ]:
#np.log(pd.Series(lst).value_counts() / sum(pd.Series(lst).value_counts())) * pd.Series(lst).value_counts() / sum(pd.Series(lst).value_counts())

In [ ]:
for i in product([1, 2,], [4, 5, 6]):
    print(i)

In [ ]:
"".join(["a", "b"])

In [ ]:
all_case_list = ["".join(map(str, case)) for case in product([0, 1, 2, 3], [0, 1], [0, 1], [0, 1])]

In [ ]:
len(all_case_list)

In [ ]:
team = "巨人"
target_flattened_triple = make_flattened_list(make_inning_triple(make_inning_list(event_df[event_df.攻撃チーム==team])))

In [3]:
files = os.listdir(SCORE_DATA_DIR)

In [7]:
pd.read_csv(os.path.join(SCORE_DATA_DIR, files[1]), encoding="cp932", index_col=0)

,1,2,3,4,5,6,7,8,9,計,安
広,0,0,0,1,0,0,0,0,0,1,4
ソ,0,0,2,1,0,1,0,0,X,4,9


In [13]:
def make_score_df(team):
    res = []
    columns = ["相手", "得点", "失点"]
    files = os.listdir(SCORE_DATA_DIR)
    for file in files:
        score_df = pd.read_csv(os.path.join(SCORE_DATA_DIR, file), encoding="cp932", index_col=0)
        if not team[0] in score_df.index:
            continue
        opponent = score_df.index[0] if score_df.index[1] == team[0] else score_df.index[1]
        team_score = score_df.loc[team[0], "計"]
        opponent_score = score_df.loc[opponent, "計"]
        res.append([opponent, team_score, opponent_score])
    return pd.DataFrame(res, columns=columns)

In [17]:
make_score_df("ロッテ")

,相手,得点,失点
0,楽,2,1
1,楽,1,8
2,ソ,3,8
3,ソ,1,6
4,日,4,5
5,日,4,5
6,楽,1,4
